# Using the Transformers library from Hugging Face to do sentiment analysis

In [1]:
!pip install transformers

from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("I've been waiting for a HuggingFace course my whole life.")
print(result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9598048329353333}]


# Supervised fine-tuning with LoRA

imports

In [2]:
!pip install datasets # install the datasets library
!pip install peft # install the peft library
!pip install evaluate # install the evaluate library

from datasets import load_dataset,  Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

import evaluate
import torch
import numpy as np




defining a model to use or fine tune

In [3]:
model_checkpoint = "distilbert-base-uncased" # using this base model for doing binary classfication because it is the smallest parameter set, can run in this machine.

#we want to fine-tune this model to do sentiment analysis on input text, for that we want to label map for positive and negavative sentinemnts.

#define label maps
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE":0, "POSITIVE":1}

#generate classification model for model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id2label,
    label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load dataset

In [4]:
#load your dataset
dataset = load_dataset("shawhin/imdb-truncated") # this dataset is currently present in hugging face datasets, it's small and for the tutorial handy, so using it.,
#contains imdb movie reviews, it has train and validation part with 1000 rows each.
# good thing about fine-tuning is you can just add a few tokenizer.trdataset of your own to test or train a model as its already has a rich resource of it's own.
dataset


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

preprocess dataset

In [5]:
#create a tokenizer, for the particular model we are using.
# models don't understand text, need to convert them to numerical data before feeding to models
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True)

#create tokenize function,
#examples is rows in dataset the training dataset has 2 columns label and text, we want to grab text from it and convert into numerical values

def tokenize_function(examples):
  #extract text
   text = examples['text']

   #tokenize and truncate, required as examples for training need to be of the same length, truncate long or pad short, or do both.
   #here truncating form left, using numpy tensor, with max length 512
   tokenizer.truncate_side = "left"
   tokenized_inputs = tokenizer(text,
                                return_tensors = "np",
                                max_length=512,
                                truncation=True)

   return tokenized_inputs

   #add pad token if not exist, tokenizer doesn't have pad tokens so adding to sequence whenever PAD is there, it's ignored by LLM
   if tokenizer.pad_token is None:
      tokenizer.add_special_tokens({'pad_token': '[PAD]'})
      model.resize_token_embeddings(len(tokenizer))

#tokenize training and validation dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

# instead of doing padding for all rows, we can dynamically PAD the rows in the datasets using collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Evaluation metrics

In [6]:
#to import the performance of the model during training

#import accuracy evaluation metrics
accuracy = evaluate.load("accuracy")

#packaging accuracy metrics as a function, one for negative and positive class, whichever is larger will become model prediction.
# define an evaluation function to pass into trainer later
def compute_metrics(eval_pred):
  predictions, labels = eval_pred # predictions here are the logits, has 2 elements + and -, evaluating which element is larger and which is larger will be the label.
  predictions = np.argmax(predictions, axis=1)
  return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

Applying untrained model to text

In [7]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - NEGATIVE
Not a fan, don't recommed. - NEGATIVE
Better than the first one. - NEGATIVE
This is not worth watching even once. - NEGATIVE
This one is a pass. - NEGATIVE


Train Model

In [8]:
peft_config = LoraConfig(task_type="SEQ_CLS", # sequence classification
                        r=4, #intrinsic rank of trainable weight matrix
                        lora_alpha=32, # learning rate
                        lora_dropout=0.01, # probability of drop out, randomly 0 internal parameters during training
                        target_modules = ['q_lin'])  # apply lora to query layer

Use config setting to update model

In [9]:
model = get_peft_model(model, peft_config) # get actual model and update it using the configuration of lora that we provided in previous step
model.print_trainable_parameters() # to see how much percentage of total parameters we actually need to model, as seen in result only 0.93% of the model will be trained, huge cost savings.

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [10]:
# hyperparameters
lr = 1e-3 # size of optimization step
batch_size = 4 # number of rows in dataset processed per optimization step
num_epochs = 10 #number of times model runs through training data

In [11]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification", # defining where model to be saved
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch", # per epoch evaluate the model parameters
    save_strategy="epoch", # per epoch save the model parameters
    load_best_model_at_end=True, # at end return best version of the model
)

In [12]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train() # from top to bottom you can see training loss decreases which is good, but validation loss is increasing too which is based as overfitting happening

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.367422,{'accuracy': 0.864}
2,0.432500,0.372985,{'accuracy': 0.882}
3,0.432500,0.615503,{'accuracy': 0.868}
4,0.235200,0.722475,{'accuracy': 0.884}
5,0.235200,0.778032,{'accuracy': 0.889}
6,0.079700,0.870770,{'accuracy': 0.89}
7,0.079700,0.923493,{'accuracy': 0.887}
8,0.029500,0.938481,{'accuracy': 0.888}
9,0.029500,1.009659,{'accuracy': 0.892}
10,0.005700,1.013857,{'accuracy': 0.891}


Trainer is attempting to log a value of "{'accuracy': 0.864}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.882}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.868}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.884}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.889}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This i

TrainOutput(global_step=2500, training_loss=0.15653376092910767, metrics={'train_runtime': 462.9679, 'train_samples_per_second': 21.6, 'train_steps_per_second': 5.4, 'total_flos': 1112883852759936.0, 'train_loss': 0.15653376092910767, 'epoch': 10.0})

Generate prediction or test

In [17]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Trained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt").to('cuda') # Ensure inputs are on the same device as the model
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Trained model predictions:
----------------------------
It was good. - POSITIVE
Not a fan, don't recommed. - NEGATIVE
Better than the first one. - POSITIVE
This is not worth watching even once. - NEGATIVE
This one is a pass. - NEGATIVE
